<a href="https://colab.research.google.com/github/nigoda/machine_learning/blob/main/24_Estimator_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to solve the Iris classification problem in TesorFlow using Estimators. An Estimator is tensorFlow's high-level representation of a complete model, and it has been designed for easy scaling and asynchronous training.

Note that in TensorFlow 2.0, the Keras API can accomplish many of these same tasks, and is believed to be an easier API to learn, If you are starting fresh, we would recommend you start with Keras. For more information about available high level APIs in TensorFlow 2.0, see Standardzing on Keras.

### **First things first**

In order to get started, you will first import TensorFlow and a number of libraries you will need.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow

import tensorflow as tf

import pandas as pd

### **The data set**

The sample program in the document builds and tests a model that classifies Iris flowers into three different species based on the size of their sepals and petals.

You will train a model using the Iris data set. The Iris data set contains four feature and one label. The four feature identify the following botanical characteristics of individual Iris flower:

*  sepal length
*  sepal width
*  petal length
*  petal width

Based on this information, you can define a few helpful constants for parsing the data:

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Next, download and parse the Iris data set using Keras and Pandas. Note that you keep distinct dataset for training and testing.

In [ ]:
train_path = tf.keras.utils.get_file(
  "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [ ]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


You can inspect your data to see that you have float feature column and one int32 label.

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


For each of the datasets, split out the labels, which the model will be trained to predict.

In [ ]:
train_y = train.pop('Species')
test_y  = test.pop('Species')

In [ ]:
# The label column has now been removed from the feature.

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


### **Overview of programming with Estimators**

Now that you have the data set up, you can define a model using a tensorFlow Estimator. An Estimator is any class derived from `tf.estimator.Estimator.` TensorFlow provides a collection of `tf.estimator (for example, LenearRegressor)` to impliment common ML algorithms. Beyond those,you may write your own custom Estimators. We recommend using pre-made Estimators when just getting started.

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

*  Creatre one or more input functions.
*  Define the model's fearture columns.
*  Instantiate an Estimator, specifying the feature columns and various hyperparameters.
*  Call one or more methods on the Estimator object, pqassing the appropriate input function as the source of the data.

let's see how those tasks are implemented for Iris classification.

### **Create input function**

You must create input function that return a `tf.data.Dataset` object which outputs the following two-element tuple:
*  features - A Python dictionary in which:
      *  Each key is the name of a feature.
      *  Each values is an array containing all of that feature's values.

*  label - An array containing the values of the label for every example.

Just to demonstrate the format of the input function, here's a simple implementation:

In [ ]:
def input_evaluation_set():
  features = {'SepalLength' : np.array([6.4, 5.0]),
              'SepalWidth' : np.array([2.8, 2.3]),
              'PetalLength' : np.array([5.6, 3.3]),
              'PetalWidth' : np.array([2.2, 1.0])}
  labels = np.array([2, 1])
  return features, labels

Your input function may generate the featurea dictionary and label list any way you like. However, we recommend using TensorFlow's Dataset API, which can parse all sorts of data.

The Dataset API can handle a lot of common cases for you. For example, using Dataset API, you can easily read in records from a large coollection of files in parallel and join them into a single stream.

To keep things simple in this example you are going to load the data with pandas, and build an input pipeline from this in-memory data:

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
  """An input funtion for training or evaluation"""
  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Shuffle and repeat if you are in training mode.
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

### **Define the feature columns**

A feature column is an object describing how the modle should use raw data from the features dictionary. When you build an Estimator model, you pass it a list of feature columns that describes each of the features you want the model to use. The `tf.feature_column` module provides many options for representing data to the model.

For Iris, The 4 raw features are numeric values, so we'll build a list of feature columns to tell the Estimator model to represent each of the four features are 32-bit floating-point values. Therefore, the code to create the feature column is:

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns =[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Feature columns can be far more sophisticated than those we're showing here. Now that you have the description of how you want the model to represent the raw features, you can build the estimator.

### Instantiate an estimator

The Iris problem is a classic classification problem. Fortunately, TensorFlow provides several pre-made classifier Estimators, including:

*  `tf.estimator.DNNClassifier` for deep models that perform multi-class classifiaction.
*  `tf.estimator.DNNLinearCombinedClassification` for wide & deep models.
*  `tf.estimator.LinearClassifier` for classification based on linear models.

For the Iris problem, `tf.estimator.DNNClassifier` seems like the best choice. Here's how you instantiated this Estimator:

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfm3lb8wl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### **Train, Evaluate, and Preedict**

Now that you have an Estimator object, you can call methods to do the folowing:

*  Train the model.
*  Evaluate the trained model.
*  Use the trained model to make predictions.



**Train the model**

Train the model by calling the estimator's train method as follows:

In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps = 5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfm3lb8wl/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 4.9171724, step = 0
INFO:tensorflow:global_step/sec: 407.798
INFO:tensorflow:loss = 2.5517285, step = 100 (0.248 sec)
INFO:tensorflow:global_step/sec: 456.358
INFO:tensorflow:loss = 1.927783, step = 200 (0.218 sec)
INFO:tensorflow:global_step/sec: 496.052
INFO:tensorflow:loss = 1.5905243, step = 300 (0.202 sec)
INFO:tensorflow:global_step/sec: 519.666
INFO:tensorflow:loss = 1.3668315, step = 400 (0.196 sec)
INFO:tensorflow:global_step/sec: 532.414
INFO:tensorflow:loss = 1.1677252, step = 500 (0.187 sec)
INFO:te

Note that you wrap up your input_fn call in a lambda to capture the arguments while providing an input function that takes no arguments, as expected by the Estimator. The steps argument tells tthe methods to stop training after a number of training steps.

**Evaluate the trained model**

Now that the model has been trained, you can get some statistics on its performance. The following code blocks evaluates the accuracy of the trained model on the test data:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-30T12:39:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfm3lb8wl/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.24682s
INFO:tensorflow:Finished evaluation at 2021-06-30-12:39:16
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.57767546, global_step = 5000, loss = 0.57767546
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpfm3lb8wl/model.ckpt-5000

Test set accuracy: 0.700



Unlike the call to train method, you did not pass the steps argumnet to evaluate. The `input_fn` for eval only yields a single epoch of data. The `eval_result` dictionary also contains the `average_loss(mean loss per samples), the loss(mean loss per mini-batch)` and the value of the estimator's `global_step(the number of training iterations it underwent).` 

**Making predication(inferring) from the training model**

You now have a trained model that produces good evaluation results. You can now use the trained model to predict the speies of an Iris flowers based on some unlabeled measurements. As with training and evaluation, you make predictions using a single function call:

In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'virginica']
predict_x = {
    'SepalLength' : [5.1, 5.9, 6.9],
    'SepalWidth' : [3.3, 3.0, 3.1],
    'PetalLength' : [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
  """An input function for prediction."""
  # Convert the input to a Dataset without labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

The predict method returna a python iterable, yielding a dictionary of prediction results for each examples. The following code prints a few predictions and their probabilities:

In [ ]:
for pred_dict, expec in zip(predictions, expected):
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
      SPECIES[class_id], 100*probability, expec))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfm3lb8wl/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (76.1%), expected "Setosa"
Prediction is "Virginica" (46.8%), expected "Versicolor"
Prediction is "Virginica" (58.6%), expected "virginica"
